In [1]:
import pandas as pd

from country_abbrev import *
from country_language import *
from pytrends.request import TrendReq

import os
import pycountry
import itertools

from googletrans import LANGCODES
import swifter

import trends_helpers
import numpy as np

In [2]:
currencies = pd.read_excel("countries-codes-and-currencies-(2020).xlsx")

In [3]:
keywords_list= list(set(currencies["Currency Name"]))
len(keywords_list)

141

In [4]:
countries_iso3 = currencies["Country Code"]
countries = [pycountry.countries.get(alpha_3=code).alpha_2 for code in countries_iso3]

In [5]:
# list of all unique languages:
unique_languages = pd.Series(list(set(list(itertools.chain(*country_language_dict.values())))), name='language')

# list of language codes from googletrans
langcodes = pd.DataFrame.from_dict(LANGCODES, orient='index', columns=['code'])
langcodes.index = langcodes.index.str.capitalize()

refugee_lang = unique_languages.to_frame().merge(langcodes, left_on='language', right_index=True, how='left')

refugee_lang.dropna(inplace=True)

refugee_lang_not_en = refugee_lang[refugee_lang['code'] != 'en']

In [6]:
translated_keyword_currency = refugee_lang_not_en['code'].swifter.apply(lambda x: trends_helpers.translate_keywords_list(lst = keywords_list, lang= x))

Pandas Apply:   0%|          | 0/83 [00:00<?, ?it/s]

In [7]:
columns = list(refugee_lang_not_en['code'])

df = pd.concat([pd.DataFrame(sublist, columns=[col]) for sublist, col in zip(translated_keyword_currency, columns)], axis=1)

df['en']=keywords_list

df.head()

,lb,id,be,am,fa,af,xh,tg,ig,ceb,...,lt,es,pl,hr,bn,ky,su,sn,pt,en
0,leone,leon,леонэ,ሊዮን,لئون,leone,leone,леоне,leone,leone,...,leonė,leona,leone,leone,লিওন,leone,leone,leone,leoa,Leone
1,südsudanesescht pond,pound sudan selatan,паўднёвасуданскі фунт,የደቡብ ሱዳን ፓውንድ,پوند سودان جنوبی,suid-soedanese pond,iponti yasemzantsi sudan,фунти судони ҷанубӣ,pound south sudan,pound sa habagatang sudan,...,pietų sudano svaras,libra sursudanesa,funt południowosudański,južnosudanska funta,দক্ষিণ সুদানিজ পাউন্ড,түштүк судан фунту,pound sudan kidul,south sudanese pound,libra sul-sudanesa,South Sudanese Pound
2,ungaresche forint,forint hungaria,венгерскі форынт,የሃንጋሪ ፎሪንት።,فورینت مجارستان,hongaarse forint,i-hungarian forint,форинти венгрия,hungarian forint,hungarian nga forint,...,vengrijos forintas,florín húngaro,węgierski forint,mađarska forinta,হাঙ্গেরিয়ান ফরিন্ট,венгриялык форинт,forint hungaria,hungarian forint,florim húngaro,Hungarian Forint
3,jordan dinar,dinar yordania,іярданскі дынар,የዮርዳኖስ ዲናር,دینار اردن,jordaanse dinar,jordanian dinar,динори иордания,dinar jordan,jordanian dinar,...,jordanijos dinaras,dinar jordano,dinar jordański,jordanski dinar,জর্ডানিয়ান দিনার,иордания динары,dinar yordania,jordanian dinar,dinar jordaniano,Jordanian Dinar
4,island krone,krona islandia,ісландская крона,የአይስላንድ ክሮና,کرون ایسلند,yslandse kroon,i-icelandic krona,кронаи исландия,iceland krone,icelandic nga krona,...,islandijos krona,corona islandesa,korona islandzka,islandska kruna,আইসল্যান্ডিক ক্রোনা,исландия кронасы,krona islandia,icelandic krona,coroa islandesa,Icelandic Krona


In [8]:
max_length = max(map(len, country_language_dict.values()))

data_padded = {key: arr + [np.nan] * (max_length - len(arr)) for key, arr in country_language_dict.items()}

langs = pd.DataFrame(data_padded)
langs = langs.T
langs = langs.reset_index()
langs = langs.rename(columns={'index': 'Country', 0:'lang1', 1:'lang2', 2:'lang3'})

# Apply the function to the 'Country' column
langs['ISO2'] = langs['Country'].apply(trends_helpers.get_iso2_country_code)

langs = langs.drop(columns=["Country"])
langs_long = pd.melt(langs, id_vars=['ISO2'], var_name='numlang', value_name='lang')
langs_long = langs_long.dropna()

In [9]:
langs_long = pd.merge(langs_long, refugee_lang_not_en, left_on="lang", right_on="language")

In [10]:
langs_long

,ISO2,numlang,lang,language,code
0,AF,lang1,Pashto,Pashto,ps
1,AL,lang1,Albanian,Albanian,sq
2,RS,lang1,Albanian,Albanian,sq
3,MK,lang2,Albanian,Albanian,sq
4,DZ,lang1,Arabic,Arabic,ar
...,...,...,...,...,...
235,IN,lang3,Telugu,Telugu,te
236,ID,lang3,Sundanese,Sundanese,su
237,NG,lang3,Igbo,Igbo,ig
238,PK,lang3,Sindhi,Sindhi,sd


In [11]:
count_files=trends_helpers.file_counter("currency_partial results/original_lang")
count_files

53

In [12]:
len(countries)

194

In [17]:
from pytrends.request import TrendReq
import time


# Set up the Google Trends API object
# pytrends = TrendReq()

results = pd.DataFrame() 

# Loop through regions and keywords

count_files=trends_helpers.file_counter("currency_partial results/original_lang")


for i, country in enumerate(langs_long["ISO2"][count_files+6:], start=count_files+6): # added +x a posteriori - empty dfs?  
    
    languagecode= langs_long["code"][i]

    for keyword in df[languagecode]:    
        print("Searching: " + str(keyword) + " in " + str(country))
        time.sleep(4)
        
        pytrends = TrendReq(tz=360, timeout=(10, 25), retries=2, backoff_factor=0.5,
            requests_args={'headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}})
        
        # Build the payload with the selected region, keyword and date range
        pytrends.build_payload(kw_list=[keyword], timeframe='all', geo=str(country))
        # Get the interest over time data for the selected keyword
        interest_over_time_df = pytrends.interest_over_time()

        # Check if the DataFrame is not empty
        if not interest_over_time_df.empty:
            interest_over_time_df= interest_over_time_df.reset_index()
            interest_over_time_df['trends_index'] = interest_over_time_df.iloc[:, 1] 
            interest_over_time_df = interest_over_time_df.drop(interest_over_time_df.columns[1], axis=1)

            interest_over_time_df = interest_over_time_df.drop(columns=["isPartial"], axis=1)
            interest_over_time_df["keyword"]=keyword
            interest_over_time_df["region"] = country
            
            # Append the results to the main dataframe
            results = pd.concat([results, interest_over_time_df])
    
    
    results_country = results[results["region"]==country]
    if not results_country.empty:
        translation = {'keyword': df[langs_long["code"][i]], 'keyword_en': df["en"]}
        translation = pd.DataFrame(translation)
        results_country = pd.merge(results_country, translation, left_on="keyword", right_on="keyword", how="left")
        file_path = 'currency_partial results/original_lang/' + str(country) + '.csv'
        results_country.to_csv(file_path, index=False)
    
    time.sleep(300) # Wait a few minutes until starting with next country 


Searching: լեոնե in JO
Searching: հարավային սուդանի ֆունտ in JO
Searching: հունգարական ֆորինտ in JO
Searching: հորդանանի դինար in JO
Searching: իսլանդական կրոն in JO
Searching: հոնդուրասի լեմպիրա in JO
Searching: հաիթիի գուրդ in JO
Searching: իրաքյան դինար in JO
Searching: քուվեյթյան դինար in JO
Searching: ճապոնական իեն in JO
Searching: սերբական դինար in JO
Searching: նեպալյան ռուփի in JO
Searching: կոստա ռիկայի քյոլն in JO
Searching: լիբիական դինար in JO
Searching: մավրիկիոսի ռուփի in JO
Searching: սեյշելյան ռուփի in JO
Searching: սոմալիի շիլլինգ in JO
Searching: բոլիվիանո in JO
Searching: ավստրալիական դոլար in JO
Searching: մալագասիկ արիարի in JO
Searching: ինդոնեզական ռուպիայով in JO
Searching: գումար in JO
Searching: նոր մանաթ in JO
Searching: նգուլտրում in JO
Searching: շվեդական կրոն in JO
Searching: նիկարագուա կորդոբա ոսկի in JO
Searching: cfa ֆրանկ in JO
Searching: գրիվնյա in JO
Searching: սաուդյան ռիալ in JO
Searching: ուգանդական շիլլինգ in JO
Searching: մարոկկոյի դիրհամ in JO


KeyError: 'region'